In [1]:
import pandas
import os
import time
from IPython.display import display, HTML
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from datetime import datetime#, timedelta
from dateutil.relativedelta import relativedelta
import re
print('ok')

/home/idies/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/idies/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ok


# Load data

In [2]:
s = time.time()
tweets_df = pandas.read_csv('tweets.csv', low_memory=False, encoding='utf-8')
e = time.time()
print('Loaded {0:,.0f} tweets in {1:,.0f} seconds!'.format(len(tweets_df), e-s))

s = time.time()
print('converting dates...')
tweets_df['publish_date'] = pandas.to_datetime(tweets_df['publish_date'])
tweets_df['harvested_date'] = pandas.to_datetime(tweets_df['harvested_date'])

tweets_df = tweets_df.set_index('tweet_id')
totalsize = len(tweets_df)
e = time.time()
print('Data loaded in {0:,.0f} seconds!'.format(e-s))
s = time.time()
usa_tweets_df = tweets_df[(tweets_df['region'] == 'United States') & (tweets_df['language'] == 'English')]
e = time.time()
print('Selected {0:,.0f} U.S.-region English-language tweets in {1:,.1f} seconds.'.format(len(usa_tweets_df), e-s))
#tweets_df.dtypes
usa_tweets_df.sample(3)

Loaded 2,973,371 tweets in 31 seconds!
converting dates...
Data loaded in 4 seconds!
Selected 1,849,687 U.S.-region English-language tweets in 1.2 seconds.


,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
tweet_id,,,,,,,,,,,,,,,
315191,4.355797e+09,SOUTHLONESTAR,Europe launches direct #rapefugee train route ...,United States,English,2016-02-18 23:50:00,2016-02-18 23:50:00,5321,5582,836,NaN,Right,0,0,RightTroll
266641,1.658421e+09,BAOBAEHAM,"#NowPlaying: ""Revolution"" Snoop Dogg feat. Oct...",United States,English,2016-11-13 09:10:00,2016-11-13 09:10:00,833,1035,2418,RETWEET,left,0,1,LeftTroll
174483,1.877493e+09,CHESPLAYSCHESS,PimpBillClinton: RT leyawn: SOCRATES: I am wis...,United States,English,2016-12-04 04:10:00,2016-12-04 04:10:00,1049,1389,16326,RETWEET,Right,0,1,RightTroll


# Display number of tweets by region

In [ ]:
#tweets_english_df = tweets_df[tweets_df['language'] == 'English']
regions_s = tweets_df.groupby('region').size().sort_values(ascending=False)
othercount = 0
htmlString = '<table>'
htmlString += '<tr><th>Region</th><th>Tweets</th><th>Percentage</th></tr>'
for thisregion, thisn in regions_s.iteritems():
    htmlString += '<tr>'
    if (thisn < 1000):
        othercount = othercount + thisn
    else:
        htmlString += '<td>' + thisregion + '</td>'
        htmlString += '<td>' + '{0:,.0f}'.format(thisn) + '</td>'
        if (thisn < (totalsize * 0.01)):
            htmlString += '<td>' + '{0:0.3%}'.format(thisn/totalsize) + '</td>'
        else:
            htmlString += '<td>' + '{0:0.1%}'.format(thisn/totalsize) + '</td>'
    htmlString += '</tr>'
htmlString += '<tr><td>{0:}</td><td>{1:,.0f}</td><td>{2:0.3%}'.format('All others', othercount, othercount/totalsize)
htmlString += '</table>'


display(HTML(htmlString))

# Display number of tweets by language

In [ ]:
#tweets_english_df = tweets_df[tweets_df['language'] == 'English']
languages_s = tweets_df.groupby('language').size().sort_values(ascending=False)
othercount = 0
htmlString = '<table>'
htmlString += '<tr><th>Language</th><th>Tweets</th><th>Percentage</th></tr>'
for thislang, thisn in languages_s.iteritems():
    htmlString += '<tr>'
    if (thisn < 8000):
        othercount = othercount + thisn
    else:
        htmlString += '<td>' + thislang + '</td>'
        htmlString += '<td>' + '{0:,.0f}'.format(thisn) + '</td>'
        if (thisn < (totalsize * 0.01)):
            htmlString += '<td>' + '{0:0.3%}'.format(thisn/totalsize) + '</td>'
        else:
            htmlString += '<td>' + '{0:0.1%}'.format(thisn/totalsize) + '</td>'
    htmlString += '</tr>'
htmlString += '<tr><td>{0:}</td><td>{1:,.0f}</td><td>{2:0.3%}'.format('All others', othercount, othercount/totalsize)
htmlString += '</table>'


display(HTML(htmlString))

# Plot number of tweets per day

In [ ]:
df = tweets_df.sample(1000)
fig, ax = plt.subplots(figsize=(15,6))

thedates = []
date1 = datetime.strptime('2014-10-01', '%Y-%m-%d')
date2 = datetime.strptime('2018-01-01', '%Y-%m-%d')
thisdate = date1
while (thisdate <= date2):
    thedates.append(thisdate)
    thisdate = thisdate + relativedelta(days=1)

thedates = [mdates.date2num(x) for x in thedates]
ax = tweets_df['publish_date'].hist(bins=thedates, color='#ff4500')

months = mdates.MonthLocator(interval = 3)
monthsFmt = mdates.DateFormatter('%b %Y')

ax.set_xlim((date1,date2))

ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
tickvalues = ax.get_yticks()
ax.yaxis.set_ticklabels('{:,.0f}'.format(x) for x in tickvalues)

ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Tweets per day', fontsize=16)
plt.title('Russian troll tweets by day', fontsize=24)

#plt.savefig('tweets_per_day.svg', format='svg')
#print('Figure saved!')
plt.show()

# Look for tweets that pretend to be Americans

In [10]:
usa_tweets_df['author'][usa_tweets_df['author'].apply(lambda x: re.search('gun', str(x).lower()) != None)].drop_duplicates()

tweet_id
221052    USA_GUNSLINGER
34212       GUNS4LIFE_ME
Name: author, dtype: object

In [ ]:
rei = re.compile('I+(\'|a)+m+\s+')
usai = re.compile('(u+\.*s+\.*a+\.)|(united states)|america+\n*')
rttag = re.compile('^RT+')

usa_tweets_df[['author', 'account_category', 'publish_date', 'content']][
    (usa_tweets_df['retweet'] == 0) &
    (usa_tweets_df['content'].apply(lambda x: re.search(rttag,str(x)) == None)) &
    (usa_tweets_df['content'].apply(lambda x: re.search(rei,str(x)) != None)) &
    (usa_tweets_df['content'].apply(lambda x: re.search(usai, str(x).lower()) != None))
].to_csv('fake_americans.csv')#.tolist()
print('File written')

# Look for tweets with conservative watchword "illegals"

In [ ]:
illegals_df = tweets_df[
    (tweets_df['region'] == 'United States') & 
    (tweets_df['language'] == 'English') &
    (tweets_df['content'].apply(lambda x: 'illegals' in str(x)))
]#.shape#.sample(10)

fig, ax = plt.subplots(figsize=(15,6))

thedates = []
date1 = datetime.strptime('2014-10-01', '%Y-%m-%d')
date2 = datetime.strptime('2018-01-01', '%Y-%m-%d')
thisdate = date1
while (thisdate <= date2):
    thedates.append(thisdate)
    thisdate = thisdate + relativedelta(days=1)

thedates = [mdates.date2num(x) for x in thedates]
ax = illegals_df['publish_date'].hist(bins=thedates, color='#ff4500')

months = mdates.MonthLocator(interval = 3)
monthsFmt = mdates.DateFormatter('%b %Y')

ax.set_xlim((date1,date2))

ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
tickvalues = ax.get_yticks()
ax.yaxis.set_ticklabels('{:,.0f}'.format(x) for x in tickvalues)

ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Tweets per day', fontsize=16)
plt.title('Russian troll tweets with the word "illegals" by day', fontsize=24)

#plt.savefig('tweets_per_day.svg', format='svg')
#print('Figure saved!')
plt.show()